# 스타벅스와 맥도날드의 드라이브스루 지점 크롤링 및 지도 시각화

## 1. 스타벅스

In [54]:
from selenium import webdriver
import warnings
warnings.simplefilter(action="ignore")
import time
import pandas as pd


driver = webdriver.Chrome("../driver/chromedriver.exe")


url = "https://www.starbucks.co.kr/store/store_map.do"
driver.get(url)
time.sleep(2)

In [2]:
driver.find_element_by_css_selector("#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a").click()
#지역검색 버튼 누르기
time.sleep(1)
driver.find_element_by_css_selector("#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a").click()
#서울 누르기
time.sleep(1)
driver.find_element_by_css_selector("#mCSB_2_container > ul > li:nth-child(1) > a").click()

In [3]:
#옵션버튼 누르기
time.sleep(1)
driver.find_element_by_css_selector("#container > div > form > fieldset > div > section > article.find_store_cont > header > p > a").click()

In [4]:
time.sleep(2)
#드라이브스루 누르기
driver.find_element_by_css_selector("dl.opt_select_dl1>dd.right>.icon02").click()

In [5]:
driver.find_element_by_css_selector("#storeMap > form > fieldset > div > ul > li.li2 > a").click()
#적용버튼 누르기

In [6]:
names = driver.find_elements_by_css_selector("#mCSB_3_container > ul > li > strong")
address = driver.find_elements_by_css_selector("#mCSB_3_container > ul > li > p")
driver
print(names[0].text,address[0].text.split("\n")[0])#전화번호까지 붙어있어서 split을 사용해서 떼네줄 수 있따.

방화DT 서울특별시 강서구 방화대로 375 (방화동)


In [7]:
data = driver.find_elements_by_css_selector("#mCSB_3_container > ul > li")
name_list = [name.get_attribute("data-name") for name in data]
lng_list = [lng.get_attribute("data-long") for lng in data]
lat_list = [lat.get_attribute("data-lat") for lat in data]
print(len(name_list),len(lng_list),len(lat_list))
print(name_list[16],lng_list[16],)

17 17 17
강동암사DT 127.130065


In [8]:
new_df = pd.DataFrame({"title":name_list,"lat":lat_list,"lng":lng_list})
new_df.head()

,title,lat,lng
0,방화DT,37.574339,126.816415
1,화곡DT,37.535149,126.863232
2,신림DT,37.480935,126.908407
3,낙성대DT,37.476169,126.965005
4,구의DT,37.5451984,127.0871587


## 2. 맥도날드

In [55]:
url = "https://www.mcdonalds.co.kr/kor/store/list.do"
driver.get(url)

In [12]:
# 모든 구의 이름을 직접 저장했습니다
real_gu = ["강남구",
"강동구",
"강북구",
"강서구",
"관악구",
"광진구",
"구로구",
"금천구",
"노원구",
"도봉구",
"동대문구",
"동작구",
"마포구",
"서대문구",
"서초구",
"성동구",
"성북구",
"송파구",
"양천구",
"영등포구",
"용산구",
"은평구",
"종로구",
"중구",
"중랑구"]

In [112]:
macs = []
for gu in real_gu:
    print(gu)
    try:
        driver.find_element_by_css_selector("#searchWord").clear()
    except:
        pass
    driver.find_element_by_css_selector("#searchWord").send_keys(gu)
    driver.find_element_by_css_selector("#searchForm > div > div > div > span:nth-child(2) > label").click()
    driver.find_element_by_css_selector("#searchForm > div > fieldset > div > button").click()
    time.sleep(1)
    
    dt_list=driver.find_elements_by_css_selector("#container > div.content > div.contArea > div > div > div.mcStore > table > tbody > tr")
    
    for data in dt_list:
        tmp = data.find_element_by_css_selector("td.tdName > dl > dt > strong > a")
        lat,lng = tmp.get_attribute("href")[19:-2].split(",")
        title = tmp.text
        print(title,lat,lng)
        text = data.text
        
        if("서울" in text and "맥드라이브" in text):
            macs.append({
                "title":title,
                "lat":lat,
                "lng":lng
            })

강남구
강남삼성DT 37.5108629 127.0473756
청담DT 37.5237111 127.0403004
압구정CGV 37.524435 127.028976
강동구
서울둔촌DT 37.5292800 127.1373340
굽은다리역DT 37.5472560 127.1429840
상일동점 37.5498037 127.1753478
강북구
미아DT 37.6180930 127.0217080
서울번동DT점 37.6211420 127.0452450
강서구
부산명지DT점 35.0974674 128.9225055
우장산DT 37.5450630 126.8373360
염창 DT점 37.5476270 126.8715000
송정역 37.561718 126.809705
등촌 DT 37.5582134 126.8597362
관악구
광진구
구의역 37.536406 127.084466
어린이대공원점 37.5472224 127.0730854
구로구
고척DT 37.4972280 126.8635360
금천구
서울시흥DT 37.4504600 126.9026210
가산비지니스센터점 37.4801705 126.8811402
가산디지털 37.4813244 126.8837789
마리오 아울렛 37.4779193 126.8872055
노원구
상계DT 37.6723950 127.0562230
도봉구
방학역DT점 37.6700480 127.0432740
쌍문DT점 37.653641 127.037193
동대문구
동작구
숭실대 37.4950710 126.9568870
노량진 37.5134312 126.9435350
중앙대 37.506553 126.958968
이수점 37.484658 126.980731
마포구
서대문구
홍제역 37.588756 126.944663
명지대점 37.5798960 126.9247250
연세대점 37.5585515 126.9367086
서초구
서초GS점 37.486518 127.009527
양재 SK DT 37.478071 127.03961
성동구
한양대 37.557201 127.04191

,title,lat,lng
0,강남삼성DT,37.5108629,127.0473756
1,청담DT,37.5237111,127.0403004
2,압구정CGV,37.524435,127.028976
3,서울둔촌DT,37.5292800,127.1373340
4,굽은다리역DT,37.5472560,127.1429840
5,미아DT,37.6180930,127.0217080
6,서울번동DT점,37.6211420,127.0452450
7,우장산DT,37.5450630,126.8373360
8,염창 DT점,37.5476270,126.8715000
9,등촌 DT,37.5582134,126.8597362


In [114]:
mac_df = pd.DataFrame(macs)
mac_df

,title,lat,lng
0,강남삼성DT,37.5108629,127.0473756
1,청담DT,37.5237111,127.0403004
2,압구정CGV,37.524435,127.028976
3,서울둔촌DT,37.5292800,127.1373340
4,굽은다리역DT,37.5472560,127.1429840
5,미아DT,37.6180930,127.0217080
6,서울번동DT점,37.6211420,127.0452450
7,우장산DT,37.5450630,126.8373360
8,염창 DT점,37.5476270,126.8715000
9,등촌 DT,37.5582134,126.8597362


In [116]:
mac_df.head()

,title,lat,lng
0,강남삼성DT,37.5108629,127.0473756
1,청담DT,37.5237111,127.0403004
2,압구정CGV,37.524435,127.028976
3,서울둔촌DT,37.5292800,127.1373340
4,굽은다리역DT,37.5472560,127.1429840


In [117]:
mac_df["brand"]="mcdonals"
new_df["brand"]="starbucks"
all_data = pd.concat([new_df,mac_df])
pd.concat([all_data.head(3),all_data.tail(3)])


,title,lat,lng,brand
0,방화DT,37.574339,126.816415,starbucks
1,화곡DT,37.535149,126.863232,starbucks
2,신림DT,37.480935,126.908407,starbucks
22,파리공원점,37.537162,126.881005,mcdonals
23,신월남부 DT점,37.5197288,126.8387402,mcdonals
24,양평SK DT,37.5309805,126.893343,mcdonals


In [118]:
import folium
import numpy as np
from tqdm import tqdm

m = folium.Map(
    location = [37.555064263408916, 126.98780752799638],
    zoom_start=11
)

for idx,rows in all_data.iterrows():
    if(rows["brand"]=="starbucks"):
        folium.Marker(
            location = [rows["lat"],rows["lng"]],
            tooltip = rows["title"],
            icon=folium.Icon(color='green',icon='star')

        ).add_to(m)
    else:
        folium.Marker(
            location = [rows["lat"],rows["lng"]],
            tooltip = rows["title"],
            icon=folium.Icon(color='red')

        ).add_to(m)        
        

m